In [7]:
import numpy as np
np.set_printoptions(linewidth = 150, precision = 4, suppress = True)

In [8]:
data = np.loadtxt('../NC-Data.csv', delimiter=',', dtype=str)
data = data[1:].astype(float)
data

array([[ 1.    , -0.0461,  0.2312, ...,  0.2704,  0.4664,  0.3672],
       [-0.0461,  1.    , -0.0671, ..., -0.0515, -0.0944, -0.0349],
       [ 0.2312, -0.0671,  1.    , ...,  0.147 ,  0.2608,  0.6313],
       ...,
       [ 0.2704, -0.0515,  0.147 , ...,  1.    ,  0.2066,  0.1538],
       [ 0.4664, -0.0944,  0.2608, ...,  0.2066,  1.    ,  0.3486],
       [ 0.3672, -0.0349,  0.6313, ...,  0.1538,  0.3486,  1.    ]])

In [9]:
nodes = np.loadtxt('../NC-K7-Trace-Nodes.csv', delimiter=',', dtype=str)
nodes = nodes[1:].astype(float)
bounds = np.loadtxt('../NC-K7-Trace-Bounds.csv', delimiter=',', dtype=str)
bounds = bounds[1:].astype(float)

In [14]:
np.argmax((nodes == 1).sum(axis=0) == 5)

60

In [21]:
bounds[:, 60]

array([4.5265, 6.1343, 7.    ])

In [23]:
selected = np.where(nodes[:, 60] == 1)[0]
selected

array([57, 82, 83, 84, 85])

In [25]:
S, V = np.linalg.eigh(data[selected, :][:, selected])
S

array([0.008 , 0.0187, 0.0738, 0.8124, 4.087 ])

In [34]:
def zd(arr):
    arr = arr.copy()
    np.fill_diagonal(arr, 0)
    return arr

In [33]:
choose = np.asarray([0, 1])
update_vec = V.T.dot(data[:, choose][selected, :]) / np.sqrt(S)[:, None]
update_vec

array([[ 0.004 ,  0.113 ],
       [-0.0758,  0.1098],
       [-0.026 , -0.1013],
       [-0.2223, -0.2183],
       [-0.0151, -0.1974]])

In [92]:
np.sort(S + abs(update_vec).sum(axis=1) * np.sqrt(S))

array([0.0185, 0.0441, 0.1084, 1.2096, 4.5166])

In [91]:
np.linalg.norm(update_vec, axis=1) * np.sqrt(S)

array([0.0101, 0.0182, 0.0284, 0.2809, 0.4002])

In [89]:
np.sort(S + np.linalg.norm(update_vec, axis=1) * np.sqrt(S))

array([0.0182, 0.0369, 0.1022, 1.0933, 4.4873])

In [90]:
np.sort(
    data[list(selected) + list(choose), list(selected) + list(choose)] + abs(zd(data[list(selected) + list(choose), :][:, list(selected) + list(choose)])).sum(axis=1)
)

array([1.2924, 2.0135, 3.0226, 4.4034, 4.4682, 4.4685, 4.5028])

In [42]:
np.linalg.eigh(data[list(selected) + list(choose), :][:, list(selected) + list(choose)])[0]

array([0.0079, 0.0184, 0.0728, 0.5835, 1.0217, 1.1576, 4.1382])

In [45]:
np.linalg.norm(update_vec[-1, :]) * S[-1]

0.8091120485444776

In [46]:
(np.linalg.norm(update_vec) ** 2 - (4.087 - 0.8124))

0.17782819635707064

In [47]:
update_vec

array([[ 0.004 ,  0.113 ],
       [-0.0758,  0.1098],
       [-0.026 , -0.1013],
       [-0.2223, -0.2183],
       [-0.0151, -0.1974]])

In [74]:
# Lower bound
np.linalg.eigh(data[list(selected), :][:, list(selected)])[0][-1] + \
(np.linalg.norm(update_vec[-2:, 0]) ** 2 - (4.087 - 0.8124) + np.sqrt(
    ((4.087 - 0.8124) + np.linalg.norm(update_vec[-2:, 0]) ** 2) ** 2
    - 4 * (4.087 - 0.8124) * update_vec[-2, 0] ** 2)
) / 2.

4.08725825415881

In [72]:
# Upper bound
np.linalg.eigh(data[list(selected), :][:, list(selected)])[0][-1] + \
(1 - (4.087 - 0.8124) + np.sqrt(
    ((4.087 - 0.8124) + 1) ** 2
    - 4 * (4.087 - 0.8124) * (1 - update_vec[-1, 0] ** 2))
) / 2.

4.087355056859358

In [60]:
np.linalg.eigh(data[list(selected) + [0], :][:, list(selected) + [0]])[0][-1]

4.087329874374427

In [80]:
all_upd = V.T.dot(data[selected, :]) / np.sqrt(S)[:, None]
all_upd[:, selected] = 0

In [83]:
np.argmax(np.linalg.norm(all_upd, axis=0))

58

In [84]:
data[57, 58]

0.9934331791632108